<a href="https://colab.research.google.com/github/raymond98tan/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Raymond_Tan_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

###Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [5]:
# Exploring our data
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [6]:
df.isna().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [7]:
# Easement is NaN's
df = df.drop('EASE-MENT',axis = 1,inplace=False)

#Apartment number is a feature with a lot of nans
df = df.drop('APARTMENT_NUMBER',axis = 1,inplace=False)

# Intepolating NaN's and filling the rest with nearest value
df = df.interpolate().ffill().bfill()

# Check
df.isna().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23040 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   BUILDING_CLASS_AT_PRESENT       23040 non-null  object 
 7   ADDRESS                         23040 non-null  object 
 8   ZIP_CODE                        23040 non-null  float64
 9   RESIDENTIAL_UNITS               23040 non-null  float64
 10  COMMERCIAL_UNITS                23040 non-null  float64
 11  TOTAL_UNITS                     23040 non-null  float64
 12  LAND_SQUARE_FEET                

In [9]:
single = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
         (df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)].reset_index(drop = True).sort_values('SALE_DATE')

single.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
1,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
22,5,OTHER,01 ONE FAMILY DWELLINGS,1,1448,1,A2,479 MAINE AVENUE,10314.0,1.0,0.0,1.0,"3,920",1850.0,1974.0,1,A2,670000,01/02/2019
20,5,OTHER,01 ONE FAMILY DWELLINGS,1,5442,145,A9,257 DOANE AVENUE,10308.0,1.0,0.0,1.0,"2,500",1392.0,1977.0,1,A9,505000,01/02/2019
19,5,OTHER,01 ONE FAMILY DWELLINGS,1,5708,35,A2,17 RATHBUN AVENUE,10312.0,1.0,0.0,1.0,"4,000",2278.0,1970.0,1,A2,552000,01/02/2019


### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [10]:
# Setting target
target = 'SALE_PRICE'
y = single[target]

y.shape

(3151,)

In [11]:
# Putting rest of features into a dataframe and dropping the target
X = single.drop('SALE_PRICE', axis=1)
X.shape

(3151, 18)

In [12]:
# Setting training data to January — March 2019 to train. Setting testing data from April 2019

cutoff = '03/31/2019'
train = single[single.SALE_DATE <= cutoff]
test = single[single.SALE_DATE > cutoff]

print(train.shape)
print(test.shape)

(2507, 19)
(644, 19)


In [13]:
X_train = train.drop(['ADDRESS','SALE_DATE'], axis=1)
y_train = train.SALE_PRICE
X_val = test.drop(['ADDRESS','SALE_DATE'], axis=1)
y_val = test.SALE_PRICE

###Do one-hot encoding of categorical features

In [14]:
single_train = single.select_dtypes(exclude='number')

single_train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
0,3,OTHER,01 ONE FAMILY DWELLINGS,1,A9,4832 BAY PARKWAY,"6,800",A9,01/01/2019
1,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,80-23 232ND STREET,"4,000",A1,01/01/2019
22,5,OTHER,01 ONE FAMILY DWELLINGS,1,A2,479 MAINE AVENUE,"3,920",A2,01/02/2019
20,5,OTHER,01 ONE FAMILY DWELLINGS,1,A9,257 DOANE AVENUE,"2,500",A9,01/02/2019
19,5,OTHER,01 ONE FAMILY DWELLINGS,1,A2,17 RATHBUN AVENUE,"4,000",A2,01/02/2019


In [15]:
# Import the class
from sklearn.preprocessing import OneHotEncoder

# Instantiate
ohe = OneHotEncoder(sparse=False)

# Fit the transformer to the categorical data
ohe.fit(single_train)

# Transform the data
single_trans = ohe.transform(single_train)

In [16]:
# Check
single_trans

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

###Do feature selection with SelectKBest

In [17]:
# Import the class
from sklearn.feature_selection import SelectKBest

# Instantiate
selector = SelectKBest(k=5)

# Fit and Transform trainer and validation
X_train = X_train.select_dtypes(include='number')
X_val = X_val.select_dtypes(include='number')

XT_train = selector.fit_transform(X_train, y_train)
XT_val = selector.transform(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [8 9] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [18]:
print(X_train.shape)
print(XT_train.shape)

(2507, 10)
(2507, 5)


In [19]:
# Create a mask to view selected features
mask = selector.get_support()

# Putting the values back into columns
X_train.columns[mask]

Index(['COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'SALE_PRICE'],
      dtype='object')

###Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

###Get mean absolute error for the test set

In [20]:
# Baseline
mean_y = y_train.mean()

print(f'Baseline mean (January — March 2019): ${mean_y:.2f}')

Baseline mean (January — March 2019): $621573.74


In [21]:
# Train Error
from sklearn.metrics import mean_absolute_error

y_pred = [y_train.mean()] * len(y_train)

mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (January — March 2019): ${mae:.2f}')

Train Error (January — March 2019): $214721.53


In [22]:
# Test Error
y_pred = [y_train.mean()] * len(y_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'Test Error (From April 2019): ${mae:.2f}')

Test Error (From April 2019): $211564.20


In [23]:
# Import model
from sklearn.linear_model import Ridge

# Instantiate class
model = Ridge(normalize = True)

# Fit model to training data from SelectKBest
model.fit(XT_train,y_train)

# Apply model to test data
y_pred2 = model.predict(XT_val)

In [24]:
# Print metrics
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE, r2_score as R2

print(f'MAE: {MAE(y_val, y_pred2):.2f}')
print(f'MSE: {MSE(y_val, y_pred2):.2f}')
print(f'R2: {R2(y_val, y_pred2):.2f}')

MAE: 101587.51
MSE: 21094636428.77
R2: 0.76
